# Clustering Crypto

In [32]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [33]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1.000000e+06
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,1.000000e+08
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,2.331000e+08
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,2.100000e+07


In [34]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [35]:
# Keep all the cryptocurrencies that are being traded.
crypto_trading_df=crypto_df.query('IsTrading==True')
crypto_trading_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00


In [36]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_trading_df[crypto_trading_df.Algorithm.notnull()]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1.000000e+09
UOS,UOS,SHA-256,True,DPoI,NaN,1.000000e+09
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1.400223e+09
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,2.100000e+07


In [37]:
# Remove the "IsTrading" column. 
crypto_trading2_df=crypto_trading_df.drop(columns=["IsTrading"], axis=1)
crypto_trading2_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,PoW,NaN,6.110000e+05
808,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1.000000e+09
UOS,UOS,SHA-256,DPoI,NaN,1.000000e+09
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [38]:
# Remove rows that have at least 1 null value.
crypto_trading3_df = crypto_trading2_df.dropna()
crypto_trading3_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
808,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141590e+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [39]:
# Keep the rows where coins are mined.
crypto_trading4_df=crypto_trading3_df.query('TotalCoinsMined>0')
crypto_trading4_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141590e+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [40]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = crypto_trading4_df[["CoinName"]]
crypto_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [41]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_trading5_df=crypto_trading4_df.drop(columns=["CoinName"], axis=1)
crypto_trading5_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,X13,PoW/PoS,2.927942e+10,3.141590e+11
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2.000000e+09
GAP,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
BDX,CryptoNight,PoW,9.802226e+08,1.400223e+09
ZEN,Equihash,PoW,7.296538e+06,2.100000e+07


In [42]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_trading5_df.copy(), columns=["Algorithm" ,"ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141590e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2.000000e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710816 -0.15287029 -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [44]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [45]:
# Create a DataFrame with the three principal components.
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca_df = pd.DataFrame(data=crypto_pca, columns=['PC1', 'PC2', 'PC3'], index=crypto_trading5_df.index)
crypto_pca_df

,PC1,PC2,PC3
42,-0.336204,1.018577,-0.592223
404,-0.319532,1.018611,-0.592690
1337,2.314313,1.699864,-0.577528
BTC,-0.143002,-1.267191,0.203484
ETH,-0.135039,-1.974774,0.426896
...,...,...,...
ZEPH,2.468507,0.956044,-0.109768
GAP,-0.334248,1.018445,-0.592250
BDX,0.323114,-2.324802,0.374849
ZEN,-0.141147,-2.049043,0.365407


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [48]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\kschn\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [108]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)
#print(predictions)
#crypto_pca_df["Class"]=model.labels_
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [119]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_trading5_df, crypto_pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(crypto_name_df['CoinName'])

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
#clustered_df = clustered_df.join((predictions['Index'])
clustered_df['Class']=predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.336204,1.018577,-0.592223,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.319532,1.018611,-0.592690,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141590e+11,2.314313,1.699864,-0.577528,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.143002,-1.267191,0.203484,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.135039,-1.974774,0.426896,1,Ethereum
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.166545,-1.151757,-0.068183,1,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.398949,1.239471,-0.477969,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.152012,-2.228107,0.363000,1,Monero
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.133480,-1.974886,0.426876,1,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.141146,-2.049043,0.365407,1,ZCash


In [123]:
class_df=clustered_df['Class']
class_df

42      0
404     0
1337    0
BTC     1
ETH     1
       ..
ZEPH    0
GAP     0
BDX     1
ZEN     1
XBC     0
Name: Class, Length: 532, dtype: int32

In [110]:
output_file_path = "clustered_df.csv"
clustered_df.to_csv(output_file_path, index=False)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [111]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC1", y="PC2", z="PC3", color="Class", symbol="Class", hover_name='CoinName', hover_data=['Algorithm'])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [112]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [113]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [114]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
MinMax=MinMaxScaler().fit_transform(clustered_df)
MinMax

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585514e-03],
       [3.14159000e-01, 2.95755050e-02],
       ...,
       [1.40022261e-03, 9.90134792e-04],
       [2.10000000e-05, 7.37027937e-06],
       [1.00000000e-06, 1.29582244e-07]])

In [125]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=MinMax, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
#plot_df = plot_df.join(clustered_df['CoinName'], index=clustered_df.index)
#clustered_df = clustered_df.join(crypto_name_df['CoinName'])
plot_df = plot_df.join(crypto_name_df['CoinName'])

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = plot_df.join(class_df)

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141590e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [127]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["Class"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class)